## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio as rio
import sklearn as sk
import scipy as sc
import seaborn as sns

import tqdm
#from tqdm import tqdm
from tqdm.notebook import tqdm # for notebooks
tqdm.pandas()

import pickle
import gc
collectAll = lambda : gc.collect(0) + gc.collect(1) + gc.collect(2)

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

from collections import namedtuple
import itertools
from itertools import product

from skimage.feature import graycomatrix
from skimage.feature import graycoprops
from math import pi

import PIL as pil
from PIL import Image, ImageDraw

import math

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


## Process each dataset

In [3]:
bands = ['wr', 'wg', 'wb', 'r', 'g', 'b', 'nir', 'red_edge', 'ndvi']#, 'chm']
props = ['mean_no_glcm', 'mean', 'contrast', 'dissimilarity', 'homogeneity', 'ASM', 'energy', 'correlation']

In [4]:
def load_rasters(prefix):
    prefix = 'geotiffs/' + prefix + '/'

    wrwgwb = prefix + 'result.tif'

    r = prefix + 'result_Red.tif'
    g = prefix + 'result_Green.tif'
    b = prefix + 'result_Blue.tif'

    nir = prefix + 'result_NIR.tif'
    red_edge = prefix + 'result_RedEdge.tif'

    #dsm = prefix + 'dsm.tif'

    raster_ds = rio.open(wrwgwb)
    #dsm_ds = rio.open(dsm)

    arr = np.stack(
        [i for i in rio.open(wrwgwb).read()[:3]] +
        [rio.open(i).read()[0] for i in [r, g, b, nir, red_edge]], axis=2)
    arr = np.nan_to_num(arr).astype('float32')

    red = arr[:,:,bands.index('r')]
    nir = arr[:,:,bands.index('nir')]
    ndvi = np.expand_dims((nir - red) / (nir + red), axis=2)
    arr = np.concatenate((arr, ndvi), axis=2)

    arr[arr == 0] = np.nan

    return prefix, raster_ds, arr

    dsm_coreg = np.empty(arr.shape[:2])
    dsm = np.nan_to_num(dsm_ds.read()[0])
    dsm[dsm < 0] = 0
    for row in tqdm(range(dsm_coreg.shape[0])):
        for col in range(dsm_coreg.shape[1]):
            x_y_spatial = raster_ds.transform * (col, row)
            dsm_row_col = dsm_ds.index(*(x_y_spatial))
            dsm_coreg[row,col] = dsm[dsm_row_col]
    dsm_coreg = np.expand_dims(dsm_coreg, axis=2)
    arr = np.concatenate((arr, dsm_coreg), axis=2)
    
    return prefix, raster_ds, arr

In [5]:
img_18Dec2020 = load_rasters('18Dec2020')

C:\Users\adil003\AppData\Local\Temp\ipykernel_5080\3111934853.py:25: RuntimeWarning: invalid value encountered in divide
  ndvi = np.expand_dims((nir - red) / (nir + red), axis=2)


In [6]:
img_18Dec2020[2].shape

(7620, 4876, 9)

In [9]:
def save_to_disk(base_path, src, arr):
    dst_profile = src.profile
    dst_profile['count'] = arr.shape[-1]
    dst_profile['dtype'] = 'float32'
    with rio.open(base_path + 'result_all_bands.tif',
                  'w',
                  **dst_profile) as dst:
        for i in range(arr.shape[-1]):
            dst.write(arr[:,:,i], i+1)

In [24]:
save_to_disk(*img_18Dec2020)

In [25]:
del img_18Dec2020

In [6]:
def load_then_save(name):
    save_to_disk(*load_rasters(name))
    gc.collect()

In [10]:
#load_then_save('10May2021')
#load_then_save('93deg')
load_then_save('183deg')

C:\Users\adil003\AppData\Local\Temp\ipykernel_21380\3111934853.py:25: RuntimeWarning: invalid value encountered in divide
  ndvi = np.expand_dims((nir - red) / (nir + red), axis=2)


In [11]:
gc.collect()

0